#### Tensors & Manipulations

In [18]:
# tensor is multidimensional array
import torch
x = torch.randn(2, 3, 4)  # create a random tensor with shape (2, 3, 4)
print(x)  # print the tensor

tensor([[[-6.7551e-01,  1.0748e+00,  1.3845e+00,  6.5353e-01],
         [-6.3221e-01,  7.9054e-01, -9.1392e-01,  7.0415e-01],
         [ 1.3968e+00, -4.4675e-01,  2.2130e+00, -8.7841e-01]],

        [[ 1.0176e-01,  8.5011e-02,  1.2939e-04,  5.2281e-01],
         [-7.2002e-01,  3.4378e-01, -9.3164e-01,  1.1351e+00],
         [-8.5246e-01, -1.7654e+00, -7.4132e-01, -1.9917e+00]]])


#### CUDA vs MPS

1. torch.cuda.is_available() checks if a CUDA-capable NVIDIA GPU is available and if PyTorch can use it. CUDA is NVIDIA’s technology for running computations on their GPUs. This is used on most Windows and Linux systems with NVIDIA GPUs.

2. torch.mps.is_available() checks if Apple’s Metal Performance Shaders (MPS) backend is available. MPS is Apple’s technology for running computations on Apple Silicon (M1, M2, M3 chips) and some Intel Macs with supported GPUs. CUDA does not work on Apple Silicon; MPS is used instead.

In [22]:
if torch.mps.is_available():
    device = torch.device("mps")
    x = torch.randn(3, 3).to(device)
    print("Running on MPS:", x.device)
else:
    print("MPS not available")

if torch.cuda.is_available():
    device = torch.device("cuda")
    x = torch.randn(3, 3).to(device)
    print("Running on CUDA:", x.device)
else:
    print("CUDA not available")

Running on MPS: mps:0
CUDA not available


#### PyTorch Parameters (`.parameters()`)

In [10]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 2)

# PyTorch makes two tensors for the weights and biases.
# Special because Pytorch marks them as things it should changes during Training.
# When we call model.parameters(), it returns these tensors.
model = MyModel()
for param in model.parameters():
    print(param.shape)
    print(param)

torch.Size([2, 4])
Parameter containing:
tensor([[-0.2013,  0.1039,  0.1993,  0.4580],
        [ 0.1079, -0.2671,  0.1110, -0.2548]], requires_grad=True)
torch.Size([2])
Parameter containing:
tensor([-0.0651, -0.4072], requires_grad=True)


In [ ]:
import torch
import torch.nn as nn

w = nn.Parameter(torch.randn(2, 2))
print(isinstance(w, nn.Parameter))

# nn.Parameter is a special kind of tensor that is automatically registered as a parameter in the module.
# It is used to define learnable parameters in a neural network.
# nn.Parameter is a subclass of torch.Tensor, so it behaves like a tensor.

# If you add this to a module, it will show up in .parameters()
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.my_weight = nn.Parameter(torch.randn(2, 2))

model = MyModel()
print(list(model.parameters()))


#### Self-Attention Layer (Transformers)

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

tokens = ["The", " ", "cat", " ", "sat", " ", "on", " ", "the", " ", "mat", "."]
n_tokens = len(tokens)
d_k = 6

# randomly initialize Q, K, V with Standard Normal distribution (mean=0, std=1)
Q = torch.randn(n_tokens, d_k) # n_tokens x d_k
K = torch.randn(n_tokens, d_k)
V = torch.randn(n_tokens, d_k)

# (n_tokens x d_k) @ (d_k x n_tokens) = (n_tokens x n_tokens)
scores = Q @ K.T 

# Values can become large, so we scale them down by the square root of d_k
# to prevent softmax from saturating
# scaling keeps variance of the dot product more consistent
# (n_tokens x n_tokens) / sqrt(d_k) = (n_tokens x n_tokens)
scaled_score = scores / (d_k ** 0.5)

# softmax to get attention weights last dimension
# For each query, softamx is applied across all keys
# converts each row to probaility distribution
# the last diimension corresponds to the keys
attn_weights = F.softmax(scaled_score, dim=-1)

# (n_tokens x n_tokens) @ (n_tokens x d_k) = (n_tokens x d_k)
# the attention weights are used to weight the values
# the result is a weighted sum of the values
output_original = attn_weights @ V

output_original

tensor([[ 0.3341, -0.5154, -1.2380, -0.2892, -0.4579, -0.2457],
        [-0.6077, -0.0793,  1.2263,  0.4887, -0.1040, -0.6966],
        [-0.2376,  0.6978, -0.2318, -0.5215,  0.0550, -0.2912],
        [-0.1975,  0.7894, -0.4018, -0.5038,  0.0247, -0.5158],
        [ 0.0433,  0.2650, -0.5138, -0.3914, -0.2075, -0.0951],
        [-0.3837,  0.6075,  0.4693, -0.1916, -0.1801, -0.0152],
        [ 0.2334, -0.6505, -1.1035, -0.1337, -0.4387, -0.4416],
        [-0.1326,  0.3576, -0.4958, -0.5872, -0.0889, -0.1419],
        [ 0.3356, -0.6253, -1.3418, -0.2246, -0.4712, -0.3358],
        [-0.1198,  0.5244, -0.3412, -0.5332, -0.0867, -0.0930],
        [-0.0488,  0.1532, -0.6066, -0.5057, -0.1759, -0.2830],
        [-0.1535,  0.2497, -0.4927, -0.4406, -0.1209, -0.2047]])